In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [0]:
import pandas as pd
print(pd.__version__)
data = pd.read_csv("/content/drive/My Drive/DA_Project/train.csv")
t_data = pd.read_csv("/content/drive/My Drive/DA_Project/test.csv")
print(data.shape)
print(t_data.shape)
#print(data)
#print(data.columns)
data = data.append(t_data, ignore_index = True)
data

1.0.3
(27486, 4)
(3535, 3)


,textID,text,selected_text,sentiment
0,a3d0a7d5ad,Spent the entire morning in a meeting w/ a ven...,my boss was not happy w/ them. Lots of fun.,neutral
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! h...,says good (or should i say bad?) afternoon!,neutral
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive
...,...,...,...,...
31016,2f8444db6c,@_shannon1234 Shannie im so sorry! didnt mean ...,NaN,negative
31017,11de8c0456,Im Slowing on My Tweets..Cuase I Lost My Phone,NaN,negative
31018,08f6036add,"Invasion of the Old Ladies has just ended, sti...",NaN,neutral
31019,27d6472b81,$#@! My nose stud fell out and I can't find it...,NaN,negative


In [0]:
def getLower(x):
  #print(x)
  if pd.isna(x):
    return ''
  return x.lower()

data = data.astype('string')
data['sentiment'] = data['sentiment'].map({'negative': -1, 'neutral': 0, 'positive': 1})
data['text'] = data['text'].map(getLower)
data['text'] = data['text'].str.replace('[^\w\s]', '')
data

,textID,text,selected_text,sentiment
0,a3d0a7d5ad,spent the entire morning in a meeting w a vend...,my boss was not happy w/ them. Lots of fun.,0
1,251b6a6766,oh good idea about putting them on ice cream,Good,1
2,c9e8d1ef1c,says good or should i say bad afternoon httpp...,says good (or should i say bad?) afternoon!,0
3,f14f087215,i dont think you can vote anymore i tried,i dont think you can vote anymore!,-1
4,bf7473b12d,haha better drunken tweeting you mean,better,1
...,...,...,...,...
31016,2f8444db6c,_shannon1234 shannie im so sorry didnt mean to...,<NA>,-1
31017,11de8c0456,im slowing on my tweetscuase i lost my phone,<NA>,-1
31018,08f6036add,invasion of the old ladies has just ended stil...,<NA>,0
31019,27d6472b81,my nose stud fell out and i cant find it loo...,<NA>,-1


In [0]:
#download punkt
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Package punkt is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [0]:
data['text'] = data['text'].apply(nltk.word_tokenize)
data

,textID,text,selected_text,sentiment
0,a3d0a7d5ad,"[spent, the, entire, morning, in, a, meeting, ...",my boss was not happy w/ them. Lots of fun.,0
1,251b6a6766,"[oh, good, idea, about, putting, them, on, ice...",Good,1
2,c9e8d1ef1c,"[says, good, or, should, i, say, bad, afternoo...",says good (or should i say bad?) afternoon!,0
3,f14f087215,"[i, dont, think, you, can, vote, anymore, i, t...",i dont think you can vote anymore!,-1
4,bf7473b12d,"[haha, better, drunken, tweeting, you, mean]",better,1
...,...,...,...,...
31016,2f8444db6c,"[_shannon1234, shannie, im, so, sorry, didnt, ...",<NA>,-1
31017,11de8c0456,"[im, slowing, on, my, tweetscuase, i, lost, my...",<NA>,-1
31018,08f6036add,"[invasion, of, the, old, ladies, has, just, en...",<NA>,0
31019,27d6472b81,"[my, nose, stud, fell, out, and, i, cant, find...",<NA>,-1


In [0]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
data['text'] = data['text'].apply(lambda x: [stemmer.stem(y) for y in x])
data

,textID,text,selected_text,sentiment
0,a3d0a7d5ad,"[spent, the, entir, morn, in, a, meet, w, a, v...",my boss was not happy w/ them. Lots of fun.,0
1,251b6a6766,"[oh, good, idea, about, put, them, on, ice, cr...",Good,1
2,c9e8d1ef1c,"[say, good, or, should, i, say, bad, afternoon...",says good (or should i say bad?) afternoon!,0
3,f14f087215,"[i, dont, think, you, can, vote, anymor, i, tri]",i dont think you can vote anymore!,-1
4,bf7473b12d,"[haha, better, drunken, tweet, you, mean]",better,1
...,...,...,...,...
31016,2f8444db6c,"[_shannon1234, shanni, im, so, sorri, didnt, m...",<NA>,-1
31017,11de8c0456,"[im, slow, on, my, tweetscuas, i, lost, my, ph...",<NA>,-1
31018,08f6036add,"[invas, of, the, old, ladi, ha, just, end, sti...",<NA>,0
31019,27d6472b81,"[my, nose, stud, fell, out, and, i, cant, find...",<NA>,-1


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

data['text'] = data['text'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()
counts = count_vect.fit_transform(data['text'])

ValueError: ignored

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)

counts = transformer.transform(counts)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, data['sentiment'], test_size=3535, shuffle=False, stratify=None)

In [0]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
import numpy as np

predicted = model.predict(X_test)

print(np.mean(predicted == y_test))

0.7114568599717115


In [0]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicted))

[[ 641  334   26]
 [ 201 1093  136]
 [  51  272  781]]


In [0]:
from sklearn.metrics import f1_score

print(f1_score(y_test, predicted, average='micro'))
print(f1_score(y_test, predicted, average='macro'))
print(f1_score(y_test, predicted, average='weighted'))

0.7114568599717114
0.7128560010998951
0.7125920837521782


In [0]:
from sklearn.metrics import jaccard_score

print(jaccard_score(y_test, predicted, average='micro'))
print(jaccard_score(y_test, predicted, average='macro'))
print(jaccard_score(y_test, predicted, average='weighted'))

0.5521405049396267
0.555104265104777
0.5546880417385829
